# Introducing `sb4dfritzlib`

I invested some time to understand how to communicate with AVM FRITZ!Box routers using the [official APIs](https://fritz.com/service/schnittstellen/). As a result, I have written my own library `sb4dfritzlib` that will eventually remove the dependency of `sb4dfritz` from the `fritzconnection` library. Not that there's anything wrong with the latter, I just wanted to know how to do it myself so that I know exactly what's going on.

Another motivating factor was the faint hope to get more accurate "real time" power measurements from energy sensors, or at least a better understanding what causes the limitations that I encountered. Recall that `fritzconnection` returns timestamps with second precision, leaving it unclear whether they were obtained by truncating or rounding. While I had little doubt that AVM is the root of this imprecision, I couldn't rule out the data processing `fritzconnection` as a cause. I can now definitively say that `fritzconnection` simply converts the XML data returned by the FRITZ!Box to appropriate Python data types. 

Anyway, here's a summary of what I did:
* I implemented the [SOAP](https://en.wikipedia.org/wiki/SOAP)-based TR-064 interface (see `sb4dfritzlib.connection`).
* I implemented the simpler AHA-HTTP interface and the required login procedure (see `sb4dfritzlib.connection`).
* I implemented various utilities for data conversion (see `sb4dfritzlib.utilities`).
* The core functionality is implemented in `sb4dfritzlib.homeauto` which provides classes `HomeAutoSystem` and `HomeAutoDevice` for object-based interaction with home automation devices.

## Comparing `sb4dfritzlib` and `fritzconnection`



In [1]:
import importlib
import sys
import json 

import fritzconnection
import sb4dfritzlib
_ = importlib.reload(sys.modules['sb4dfritzlib'])

from sb4dfritzlib.connection import FritzUser
from sb4dfritzlib.homeauto import HomeAutoSystem

CONFIG_FILE = "..\\..\\_private_files\\sb4dfritz_secrets.ini"
with open(CONFIG_FILE,"r") as file:
    CONFIG = json.load(file)
# extract login data
USER = CONFIG['login']['user']
PWD = CONFIG['login']['pwd']
IP = CONFIG['login']['ip']


Here's how to connect to the home automation system using `sb4dfritzlib`:

In [2]:
# Create FritzUser from credentials
sb4d_user = FritzUser(USER, PWD, IP)
# Connect to home automation system
sb4d_home_auto_system = HomeAutoSystem(sb4d_user)
# List registered devices
print("Registered home automation devices:")
for device in sb4d_home_auto_system.devices:
    print(" -", device)

Registered home automation devices:
 - l'angolo del caffè (AVM FRITZ!Smart Energy 200)
 - radiatore nel soggiorno (AVM FRITZ!Smart Thermo 301)
 - radiatore musicale (AVM FRITZ!Smart Thermo 301)
 - radiatore nella cucina (AVM FRITZ!Smart Thermo 301)
 - TV etc (AVM FRITZ!Smart Energy 200)


Under the hood, the instantiation of `HomeAutoSystem` first log in to obtains a valied session id, then asks for a list of connected devices, and creats a `HomeAutoDevice` object for each device. 

In my case, the first device is a smart plug which functions as an on/off switch for my espresso machine. Let's check it's current power consumption:

In [3]:
# Give coffe machine a dedicated variable
sb4d_caffe = sb4d_home_auto_system.devices[0]
sb4d_tv = sb4d_home_auto_system.devices[4]


In [4]:
# Get latest power measurement together with timing information
power_record = sb4d_caffe.get_latest_power_record()
print("Current power consumption and timing information:")
for key, val in power_record.items():
    print(f" - {key:15s} {val}")

Current power consumption and timing information:
 - power           3.29
 - datatime        2025-08-21 09:49:09
 - starttime       2025-08-21 09:50:27.770196
 - endtime         2025-08-21 09:50:28.598014
 - duration        0.827818
 - latency         79.598014
 - offset          -78.770196


As you can guess, the smart plug reported 0.0 W as `power` at the time `datatime`. 
The parameters `starttime` and `endtime` record the times that the network request to obtain the data was sent and received, respectively.
Lastly, `duration` is the total request time in seconds while `latency` and `offset` compare the `endtime` and `starttime` to `datatime`, respectively.
The large values are typical if no request has been made recently. 

At this point, it should not go unnotices that a latency of over 50 seconds, meaning that the measurement was over 50 seconds ago, can hardly be considered "real time". 

Let's make a few more requests to see what happens over time.

In [5]:
OUTPUT_TEMPLATE = """{:4s} Request: {:s} | Power: {:6.1f} W | Timestamp: {:s} | Duration: {:0.2f} | Latency: {:6.2f}"""

In [ ]:
for k in range(25):
    data = sb4d_caffe.get_latest_power_record()
    output = OUTPUT_TEMPLATE.format(
        f"({k+1})",
        data['starttime'].strftime("%H:%M:%S.%f"),
        data['power'],
        data['datatime'].strftime("%H:%M:%S"),
        data['duration'],
        data['latency'],
    )
    print(output)


From the above results, we can take away a few thing:

* Request normally take around 0.75 to 0.8 seconds, occasionally a little longer.
* The timestamp jumps 10 second interval. This means that power is measured only every 10 seconds. It also means that the **data requests *do not* trigger a measurements!** Measurements are taken periodically at times unrelated to the requests.
* On repeated requests, the latency does not exceed 10 seconds by much. This indicates some undocumented "sleep mode" which reduces the update frequency in longer time stretches without data requests.

If you're interested in real time information, this is bad news. A random request will give you a power value recorded some time in the last ~12.5 seconds, if you're lucky. Again, I would not consider this as "real time". After a little thought, you should realize that,  given the network latency (reflected in the request duration) and the second presicion in the timestamp, it's impossible to determine the time of measurement precisely from the provided data. There is always going to be some latency, and in bad cases you cannot expect to do better than ~2 seconds of latency. 

My personal motivation was to switch off my coffee machine only when it's idle (indicated by a power consumption below 5 W). The problem is that the machine has irregular heating cycles, even if it's not actively being used. 

**Problem:** How to time the data requests to guarantee minimal possible latency?



In [6]:
power_monitor = []

In [7]:
k=0
while True:
    k+=1
    data = sb4d_caffe.get_latest_power_record()
    power_monitor.append(data)
    output = OUTPUT_TEMPLATE.format(
        f"({k})",
        data['starttime'].strftime("%H:%M:%S.%f"),
        data['power'],
        data['datatime'].strftime("%H:%M:%S"),
        data['duration'],
        data['latency'],
    )
    print(output)


(1)  Request: 09:51:00.844969 | Power:    3.3 W | Timestamp: 09:50:57 | Duration: 0.83 | Latency:   4.67
(2)  Request: 09:51:01.671909 | Power:    3.3 W | Timestamp: 09:50:57 | Duration: 0.80 | Latency:   5.47
(3)  Request: 09:51:02.474250 | Power:    3.3 W | Timestamp: 09:50:57 | Duration: 1.27 | Latency:   6.74
(4)  Request: 09:51:03.744075 | Power:    3.3 W | Timestamp: 09:50:57 | Duration: 0.82 | Latency:   7.56
(5)  Request: 09:51:04.564214 | Power:    3.3 W | Timestamp: 09:50:57 | Duration: 0.82 | Latency:   8.39
(6)  Request: 09:51:05.386601 | Power:    3.3 W | Timestamp: 09:50:57 | Duration: 0.82 | Latency:   9.21
(7)  Request: 09:51:06.208129 | Power:    3.3 W | Timestamp: 09:50:57 | Duration: 0.92 | Latency:  10.12
(8)  Request: 09:51:07.124101 | Power:    3.3 W | Timestamp: 09:50:57 | Duration: 0.83 | Latency:  10.95
(9)  Request: 09:51:07.951681 | Power:    3.3 W | Timestamp: 09:50:57 | Duration: 0.88 | Latency:  11.84
(10) Request: 09:51:08.836661 | Power:    3.3 W | Times

KeyboardInterrupt: 

In [8]:
L = len(power_monitor)

jumps = []
for k in range(L-1):
    if power_monitor[k+1]['datatime'] != power_monitor[k]['datatime']:
        jumps.append(power_monitor[k+1])

for data in jumps:
    output = OUTPUT_TEMPLATE.format(
        f"({k})",
        data['starttime'].strftime("%H:%M:%S.%f"),
        data['power'],
        data['datatime'].strftime("%H:%M:%S"),
        data['duration'],
        data['latency'],
    )
    print(output)


(151) Request: 09:51:08.836661 | Power:    3.3 W | Timestamp: 09:51:07 | Duration: 0.81 | Latency:   2.65
(151) Request: 09:51:18.942047 | Power: 1446.0 W | Timestamp: 09:51:17 | Duration: 0.87 | Latency:   2.82
(151) Request: 09:51:28.850305 | Power:  865.2 W | Timestamp: 09:51:27 | Duration: 0.81 | Latency:   2.66
(151) Request: 09:51:38.159689 | Power:    3.3 W | Timestamp: 09:51:37 | Duration: 0.84 | Latency:   2.00
(151) Request: 09:51:48.352366 | Power:    3.4 W | Timestamp: 09:51:47 | Duration: 1.17 | Latency:   2.52
(151) Request: 09:51:58.224361 | Power:    3.4 W | Timestamp: 09:51:57 | Duration: 0.92 | Latency:   2.14
(151) Request: 09:52:08.208921 | Power:    3.3 W | Timestamp: 09:52:07 | Duration: 0.87 | Latency:   2.08
(151) Request: 09:52:18.701857 | Power:    3.4 W | Timestamp: 09:52:17 | Duration: 0.90 | Latency:   2.60
(151) Request: 09:52:28.979777 | Power:  152.3 W | Timestamp: 09:52:27 | Duration: 0.83 | Latency:   2.81
(151) Request: 09:52:38.858042 | Power:  314.2

Let me try if it's possible to send request twice every second using `threading`. 

In [ ]:
from threading import Thread
from time import sleep

def caffe_power(k):
    data = sb4d_caffe.get_latest_power_record()
    output = OUTPUT_TEMPLATE.format(
        f"({k})",
        data['starttime'].strftime("%H:%M:%S.%f"),
        data['power'],
        data['datatime'].strftime("%H:%M:%S"),
        data['duration'],
        data['latency'],
    )
    print(output)

for k in range(40):
    Thread(target=caffe_power, args=[k]).start()
    sleep(0.5)

(0)  Request: 10:01:59.902715 | Power:    3.4 W | Timestamp: 10:00:39 | Duration: 0.85 | Latency:  81.75
(1)  Request: 10:02:00.404695 | Power:    3.4 W | Timestamp: 10:00:39 | Duration: 1.13 | Latency:  82.53
(2)  Request: 10:02:00.907294 | Power:    3.4 W | Timestamp: 10:00:39 | Duration: 1.54 | Latency:  83.45
(3)  Request: 10:02:01.409151 | Power:    3.4 W | Timestamp: 10:00:39 | Duration: 1.91 | Latency:  84.32
(4)  Request: 10:02:01.910619 | Power:    3.4 W | Timestamp: 10:00:39 | Duration: 2.61 | Latency:  85.52
(5)  Request: 10:02:02.412251 | Power:    3.4 W | Timestamp: 10:00:39 | Duration: 3.00 | Latency:  86.41
(6)  Request: 10:02:02.914864 | Power:    3.4 W | Timestamp: 10:00:39 | Duration: 3.30 | Latency:  87.22
(7)  Request: 10:02:03.417265 | Power:    3.4 W | Timestamp: 10:00:39 | Duration: 3.59 | Latency:  88.01
(8)  Request: 10:02:03.918717 | Power:    3.4 W | Timestamp: 10:00:39 | Duration: 4.15 | Latency:  89.07
(9)  Request: 10:02:04.419652 | Power:    3.4 W | Times

(22) Request: 10:02:10.955117 | Power:    3.4 W | Timestamp: 10:02:09 | Duration: 9.06 | Latency:  11.01
(23) Request: 10:02:11.456496 | Power:    3.4 W | Timestamp: 10:02:09 | Duration: 9.38 | Latency:  11.84
(24) Request: 10:02:11.958359 | Power:    3.3 W | Timestamp: 10:02:19 | Duration: 9.69 | Latency:   2.64
(25) Request: 10:02:12.460383 | Power:    3.3 W | Timestamp: 10:02:19 | Duration: 10.02 | Latency:   3.48
(26) Request: 10:02:12.962111 | Power:    3.3 W | Timestamp: 10:02:19 | Duration: 10.32 | Latency:   4.28
(27) Request: 10:02:13.464989 | Power:    3.3 W | Timestamp: 10:02:19 | Duration: 10.64 | Latency:   5.10
(28) Request: 10:02:13.966647 | Power:    3.3 W | Timestamp: 10:02:19 | Duration: 10.91 | Latency:   5.87
(29) Request: 10:02:14.468179 | Power:    3.3 W | Timestamp: 10:02:19 | Duration: 11.20 | Latency:   6.67
(30) Request: 10:02:14.969830 | Power:    3.3 W | Timestamp: 10:02:19 | Duration: 11.50 | Latency:   7.47
(31) Request: 10:02:15.472110 | Power:    3.3 W |

This is clearly not working. The increasing duration indicates taht the FRITZ!Box is not handling the requests properly.

In [ ]:
sb4d_tv.switch_off_when_idle(status_messages='console', debug_mode=True)

3.79 0.847651 7.763409
3.71 0.846818 2.081194
3.64 0.848897 2.66714
Ideal state detected: [3.71, 3.64] [0.846818, 0.848897] [2.081194, 2.66714]


In [5]:
sb4d_caffe.switch_off_when_idle(status_messages='console', debug_mode=True)

Switch is already off. Nothing to do.


In [8]:
sb4d_tv.ain, sb4d_tv.sid

('116300073749', 'df864dc2c762661d')